Importing the Dependencies

In [3]:
import os
import json

from zipfile import ZipFile
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

Data Collection - Kaggle API

In [4]:
kaggle_dictionary = json.load(open('kaggle.json'))

In [5]:
# setup kaggle credentials as environment variables
os.environ['KAGGLE_USERNAME'] = kaggle_dictionary['username']
os.environ['KAGGLE_KEY'] = kaggle_dictionary['key']

In [6]:
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
License(s): other
  0% 0.00/25.7M [00:00<?, ?B/s]
100% 25.7M/25.7M [00:00<00:00, 956MB/s]


In [7]:
!ls

imdb-dataset-of-50k-movie-reviews.zip  kaggle.json  sample_data


In [8]:
# unzip the dataset file
with ZipFile('imdb-dataset-of-50k-movie-reviews.zip', 'r') as zip_ref:
  zip_ref.extractall()

In [9]:
!ls # to see the extracted file

'IMDB Dataset.csv'			 kaggle.json
 imdb-dataset-of-50k-movie-reviews.zip	 sample_data


Loading the Dataset

In [10]:
data = pd.read_csv('IMDB Dataset.csv')

In [11]:
data.shape

(50000, 2)

In [12]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [13]:
data.tail() # for printing the last five rows

,review,sentiment
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative
49999,No one expects the Star Trek movies to be high...,negative


In [14]:
# How is the data is distributed?
data['sentiment'].value_counts()

# there is no class imbalance

,count
sentiment,
positive,25000
negative,25000


In [15]:
data.replace({"sentiment": {"positive": 1, "negative": 0}}, inplace = True)

<ipython-input-15-2164277587>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data.replace({"sentiment": {"positive": 1, "negative": 0}}, inplace = True)


In [16]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [17]:
# split data into training data and test data
train_data, test_data = train_test_split(data, test_size = 0.2, random_state = 42)

In [18]:
print(train_data.shape)
print(test_data.shape)

(40000, 2)
(10000, 2)


Data Pre-Processing

In [19]:
y_trian = train_data['sentiment']
y_test = test_data['sentiment']

LSTM - Long Short Term Memory

In [ ]:
from transformers import BertTokenizer, BertModel
import torch
import numpy as np
from tqdm import tqdm

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
bert = BertModel.from_pretrained("bert-base-uncased")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
bert.to(device)
bert.eval()

batch_size = 128
max_len = 200
texts = list(train_data['review'])
X_embed = []

for i in tqdm(range(0, len(texts), batch_size), desc="Embedding Batches"):
    batch_texts = texts[i:i+batch_size]
    inputs = tokenizer(batch_texts, return_tensors="pt", padding='max_length', truncation=True, max_length=max_len)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = bert(**inputs)
    batch_embeddings = outputs.last_hidden_state.cpu().numpy()  # shape: (batch_size, max_len, 768)
    X_embed.extend(batch_embeddings)

X_embed = np.array(X_embed)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Embedding Batches:  45%|████▌     | 141/313 [04:31<05:22,  1.87s/it]

In [ ]:
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense

model = Sequential()
model.add(LSTM(128, input_shape=(max_len, 768), dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

model(np.zeros((1, 200)).astype(np.int32))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()

In [ ]:
# trainig the model
model.fit(X_embed, y_trian, epochs = 5, batch_size = 64, validation_split = 0.2)

Model Evaluation

In [ ]:
loss, accuracy = model.evaluate(x_test, y_test)
print(f"Test loss: {loss}")
print(f"Test accuracy: {accuracy}")

Building Predictive System

In [ ]:
def predict_sentiment(review):
  # tokenize and pad the review text
  sequence = tokenizer.texts_to_sequences([review])
  padded_sequence = pad_sequences(sequence, maxlen = 200)

  prediction = model.predict(padded_sequence)

  sentiment = 'positive' if prediction[0][0] > 0.5 else 'negative'

  return sentiment

In [ ]:
# example usage

new_review = "The movie was not only good, but the acting was also amazing!"
sentiment = predict_sentiment(new_review)
print(f"The sentiment of the review is: {sentiment}")